In [1]:
import torch
import os
import pickle
from tqdm import tqdm   
from torch.utils.data import DataLoader, ConcatDataset

from utils.model_train import train
from utils.processing import load_data, process_data, augment_data, to_tensors, split_batch, incremental_save
from utils.quickdraw_cnn import QuickDrawCNN_V1, QuickDrawCNN_V2

In [2]:
image_dir = "data/numpy_bitmap/"
categories = os.listdir(image_dir)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
labels_map = {label: i for i, label in enumerate(set(categories))}

In [3]:
def pipeline(image_dir: str, categories: list, device: torch.device, labels_map: dict):
    
    '''
    A pipeline function to run the entire process of loading, processing, augmenting, and splitting the data onto each category at a time.
    '''
    
    train_datasets, test_datasets, val_datasets = [], [], []

    for cat in tqdm(categories, desc="Processing categories"):
        
        try:
            # load, process, augment, and split the data
            features, label = load_data(image_dir, cat, file_standardize=False)
            features, label = process_data(features, label)
            # features, label = augment_data(features, label, rot=0, h_flip=False, v_flip=False)
            features, labels = to_tensors(features, label, labels_map, device=device)
            
            # split the data into train, test, and validation sets
            train_loader, test_loader, val_loader = split_batch(features, labels, batch_size=32)
            train_datasets.append(train_loader.dataset)
            test_datasets.append(test_loader.dataset)
            val_datasets.append(val_loader.dataset)
        except RuntimeError as e:
            print(f"Error on {cat}: {e}")
            
        torch.cuda.empty_cache()
        
    return train_datasets, test_datasets, val_datasets

## $$ \text{Error seen here} $$ 
$$ \text{Functions used in "utils/processing"} $$

In [4]:
train_datasets, test_datasets, val_datasets = pipeline(image_dir, categories, device, labels_map)

Processing categories:  16%|█▌        | 56/345 [02:42<19:41,  4.09s/it]

Error on banana.npy: CUDA out of memory. Tried to allocate 738.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 24.22 GiB is allocated by PyTorch, and 1.37 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Processing categories:  17%|█▋        | 58/345 [02:48<17:20,  3.62s/it]

Error on the_great_wall_of_china.npy: CUDA out of memory. Tried to allocate 462.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 24.24 GiB is allocated by PyTorch, and 1.35 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Processing categories:  17%|█▋        | 59/345 [02:51<16:31,  3.47s/it]

...

Error on bus.npy: CUDA out of memory. Tried to allocate 498.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 24.51 GiB is allocated by PyTorch, and 1.37 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Processing categories:  19%|█▉        | 65/345 [03:07<12:54,  2.77s/it]

Error on bush.npy: CUDA out of memory. Tried to allocate 290.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 24.74 GiB is allocated by PyTorch, and 1.13 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Processing categories:  19%|█▉        | 65/345 [03:10<13:40,  2.93s/it]


KeyboardInterrupt: 

In [ ]:
# concat dataloaders
train_loader = DataLoader(ConcatDataset(train_datasets), batch_size=64, shuffle=True)
test_loader = DataLoader(ConcatDataset(test_datasets), batch_size=64, shuffle=False)
val_loader = DataLoader(ConcatDataset(val_datasets), batch_size=64, shuffle=False)

In [5]:
model = QuickDrawCNN_V2().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
try:
    train_loss, val_loss, train_acc, val_acc = train(model=model,
                                                     train_loader=train_loader,
                                                     val_loader=val_loader,
                                                     epochs=10,
                                                     criterion=criterion,
                                                     optimizer=optimizer,
                                                     device=device)
except KeyboardInterrupt:
    print("Training interrupted manually. Saving current model state...")
    torch.save(model.state_dict(), "model_state.pth")
    print("Model state saved.")


Epoch 1/10


Validating: 100%|██████████| 391/391 [00:00<00:00, 603.07batch/s, acc=100, loss=5.66e-5]



Epoch: 1 | Train Loss: 0.07184 | Train Acc: 97.47% | Val Loss: 0.04996 | Val Acc: 98.17%

Epoch 2/10


Validating: 100%|██████████| 391/391 [00:00<00:00, 614.43batch/s, acc=100, loss=2.68e-7] 



Epoch: 2 | Train Loss: 0.04642 | Train Acc: 98.29% | Val Loss: 0.05436 | Val Acc: 98.05%

Epoch 3/10


Validating: 100%|██████████| 391/391 [00:00<00:00, 610.72batch/s, acc=100, loss=1.16e-6] 


Epoch: 3 | Train Loss: 0.03969 | Train Acc: 98.51% | Val Loss: 0.05468 | Val Acc: 98.00%
Early stopping at epoch 3


In [7]:
parent_path = 'saves'
model_path = f'{parent_path}/model'
var_path = f'{parent_path}/vars'

varaibles_saved = incremental_save(var_path)
with open(varaibles_saved, "wb") as f:
    pickle.dump(labels_map, f)
    pickle.dump(train_loss, f)
    pickle.dump(val_loss, f)
    pickle.dump(train_acc, f)
    pickle.dump(val_acc, f)

model_saved = incremental_save(model_path, data=model)

Path: saves/vars.0.pth
Saved to saves/model.0.pth.
